In [1]:
import csv
from openpyxl import Workbook
from openpyxl import load_workbook

In [2]:
white_list = ['অ','আ','ই','ঈ','উ','ঊ','ঋ','এ','ঐ','ও','ঔ','ক','খ','গ','ঘ','ঙ','চ','ছ','জ','ঝ','ঞ','ট','ঠ','ড',
              'ঢ','ণ','ত','থ','দ','ধ','ন','প','ফ','ব','ভ','ম','য','র','ল','শ','ষ','স','হ','ড়','ঢ়','য়','ৎ','ক্ষ',
              'ঁ','ং','ঃ','়','া','ি','ী','ু','ূ','ৃ','ৄ','ে','ৈ','ো','ৌ','্','ৗ','ৠ','ৢ','ৣ','ৰ','ৱ','০','১','২',
              '৩','৪','৫','৬','৭','৮','৯','।','?','!',',',':',';','-',' ']

In [3]:
special_char_list = ['।','?','!',',',':',';','-',' ']

In [4]:
wb = Workbook()

In [5]:
path = 'D:\EDUCATION\Deep Learning\Jupyter Notebook\Twitter Bangla Collection\Tweets_July_Bangla_Raw\\'

In [6]:
file_name = 'Tweet_July_2018_1'

In [7]:
file_extension = '.xlsx'

In [8]:
tweet_year = '2018'

In [9]:
wb = load_workbook(path+file_name+file_extension)

In [10]:
ws = wb.active

In [11]:
tuple_cells = ws['A']

In [12]:
clean_values = []
for cell_num in range(1, len(tuple_cells)+1):
    cell_date = 'A'+str(cell_num)
    cell_tweet = 'B'+str(cell_num)
    filtered_string = ''
    skip_space_before_special_char = False
    
    if ws[cell_tweet].value is not None:        
        #Tweet data cleaning::
        multi_same_special_char = ''
        for char in ws[cell_tweet].value:
            if char in white_list and char not in special_char_list and multi_same_special_char!=' ' and\
            multi_same_special_char in special_char_list:
                filtered_string += ' '
                filtered_string += char
                multi_same_special_char = ''
            elif char in white_list and char not in special_char_list:
                filtered_string += char
                multi_same_special_char = ''
            elif char in special_char_list and char!=multi_same_special_char:
                filtered_string += char
                multi_same_special_char = char
            elif char==multi_same_special_char:
                continue
            elif char is '\n' and multi_same_special_char in special_char_list and multi_same_special_char!=' ':
                filtered_string += ' '
                multi_same_special_char = ' '
            elif char is '\n' and (multi_same_special_char=='' or multi_same_special_char==' '):
                filtered_string += '; '
                multi_same_special_char = ' '
                
        filtered_string = filtered_string.replace(' ;', ';')
        
        #Date formatting::
        date = ''
        for char in ws[cell_date].value:
            if char==' ':
                date += '-'
            else:
                date += char
        date = date + '-' + tweet_year
        
        clean_values.append((date, filtered_string))

In [13]:
i = 100
print(clean_values[i],'\n',clean_values[-1*i])

('Jul-30-2018', 'ইদানীং মনে হয় এত কষ্ট কইরা এদের এত মিথ্যা বলার দরকারটা কি?; কইলেই হয় যা করসি, করসি। পারলে ঠ্যাকা নাইলে অফ যা। বাস্তবে তো তাই হইতেসে।') 
 ('Jul-22-2018', 'এক্সকিউজমি; সাইডপ্লিজ; এই মাত্রই ল্যান্ডিং খেয়েছে আমার দু চোখের পাতায় গভীর ঘুম; শুভরাত্রি')


In [14]:
import mysql.connector

In [15]:
host_name = 'localhost'
username = 'root'
password = ''
database = 'sentimentdb'

In [16]:
mydb = mysql.connector.Connect(host=host_name, user=username, passwd=password)
print(mydb)

In [17]:
mycursor = mydb.cursor()
print(mycursor)

MySQLCursor: (Nothing executed yet)


In [18]:
sql = 'CREATE DATABASE IF NOT EXISTS ' + database

In [19]:
mycursor.execute(sql)

In [20]:
sql = 'USE ' + database

In [21]:
mycursor.execute(sql)

In [22]:
sql = '''CREATE TABLE IF NOT EXISTS tweets (
    tweet_id INT AUTO_INCREMENT PRIMARY KEY,
    label_1 VARCHAR(100),
    label_2 VARCHAR(100),
    tweet TEXT NOT NULL,
    tweet_date VARCHAR(20) NOT NULL,
    labeller_1 INT,
    labeller_2 INT,
    labeling_date_1 TIMESTAMP ON UPDATE CURRENT_TIMESTAMP,
    labeling_date_2 TIMESTAMP ON UPDATE CURRENT_TIMESTAMP
    )CHARACTER SET utf8 COLLATE utf8_general_ci
'''

In [23]:
mycursor.execute(sql)

In [24]:
sql = 'INSERT INTO tweets (tweet_date, tweet) VALUES (%s, %s)'

In [25]:
total_inserted_rows = 0
for value in clean_values:
    mycursor.execute(sql, value)
    total_inserted_rows += 1
mydb.commit()
print(total_inserted_rows, 'rows have been insetred!')

9998 rows have been insetred!


In [26]:
sql = 'SELECT COUNT(*) FROM tweets'
mycursor.execute(sql)
total_entry = mycursor.fetchall()
print('Total no. of rows =', total_entry[0][0])

Total no. of rows = 9998


In [27]:
print(type(total_entry))
print(type(total_entry[0]))
print(type(total_entry[0][0]))

<class 'list'>
<class 'tuple'>
<class 'int'>


In [28]:
def drop_database():
    mycursor.execute('DROP DATABASE tweetdb')
#drop_database()

In [29]:
def drop_table():
    mycursor.execute('DROP TABLE tweets')
#drop_table()